In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
PROCESSED_PATH = '../data/processed/'

In [3]:
df = pd.read_csv(f'{PROCESSED_PATH}/transactions.csv')
df.head()

,Date,Description,Withdrawals ($),Deposits ($),Balance ($)
0,"Jan 01, 2025",NaN,OpeningBalance,NaN,22.01
1,"Jan 02, 2025",Withdrawal 25991738 Free Interac E Transfer,8.00,NaN,14.01
2,"Jan 02, 2025",MB Transfer from 232001691155,NaN,60.00,74.01
3,"Jan 02, 2025",Point of sale purchase Apos Driver Services Va...,15.00,NaN,59.01
4,"Jan 02, 2025",Point of sale purchase Apos Driver Services Va...,10.00,NaN,49.01


#### Data Exploration

In [4]:
df.shape

(618, 5)

In [5]:
df.columns

Index(['Date', 'Description', 'Withdrawals ($)', 'Deposits ($)',
       'Balance ($)'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             618 non-null    object
 1   Description      605 non-null    object
 2   Withdrawals ($)  509 non-null    object
 3   Deposits ($)     109 non-null    object
 4   Balance ($)      618 non-null    object
dtypes: object(5)
memory usage: 24.3+ KB


In [7]:
df.isnull().sum()

Date                 0
Description         13
Withdrawals ($)    109
Deposits ($)       509
Balance ($)          0
dtype: int64

In [8]:
renames = {
    'Date':'date',
    'Description':'description',
    'Withdrawals ($)':'withdrawals',
    'Deposits ($)':'deposits',
    'Balance ($)':'balance'
}
df = df.rename(columns=renames)
df.columns

Index(['date', 'description', 'withdrawals', 'deposits', 'balance'], dtype='object')

In [9]:
df.loc[df['withdrawals'] == 'OpeningBalance', 'description'] = 'Opening Balance'
df.loc[df['withdrawals'] == 'OpeningBalance', 'withdrawals'] = np.nan

df.loc[df['withdrawals'] == 'ClosingBalance', 'description'] = 'Closing Balance'
df.loc[df['withdrawals'] == 'ClosingBalance', 'withdrawals'] = np.nan

In [10]:
df['withdrawals'] = df['withdrawals'].str.replace(',','')
df['withdrawals'] = df['withdrawals'].str.replace('.','')
df['withdrawals'] = df['withdrawals'].str.replace('$','')

df['deposits'] = df['deposits'].str.replace(',','')
df['deposits'] = df['deposits'].str.replace('.','')
df['deposits'] = df['deposits'].str.replace('$','')

df['balance'] = df['balance'].str.replace(',','')
df['balance'] = df['balance'].str.replace('.','')
df['balance'] = df['balance'].str.replace('$','')

df['withdrawals'] = df['withdrawals'].astype(np.float32)
df['deposits'] = df['deposits'].astype(np.float32)
df['balance'] = df['balance'].astype(np.float32)
df['date'] = pd.to_datetime(df['date'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         618 non-null    datetime64[ns]
 1   description  618 non-null    object        
 2   withdrawals  484 non-null    float32       
 3   deposits     109 non-null    float32       
 4   balance      618 non-null    float32       
dtypes: datetime64[ns](1), float32(3), object(1)
memory usage: 17.0+ KB


In [11]:
df.head()

,date,description,withdrawals,deposits,balance
0,2025-01-01,Opening Balance,NaN,NaN,2201.0
1,2025-01-02,Withdrawal 25991738 Free Interac E Transfer,800.0,NaN,1401.0
2,2025-01-02,MB Transfer from 232001691155,NaN,6000.0,7401.0
3,2025-01-02,Point of sale purchase Apos Driver Services Va...,1500.0,NaN,5901.0
4,2025-01-02,Point of sale purchase Apos Driver Services Va...,1000.0,NaN,4901.0


In [12]:
df.to_csv(f"{PROCESSED_PATH}/clean_df.csv", index=False)

#### Feature Engineering

In [13]:
df_clean = pd.read_csv(f"{PROCESSED_PATH}/clean_df.csv")
df.head()

,date,description,withdrawals,deposits,balance
0,2025-01-01,Opening Balance,NaN,NaN,2201.0
1,2025-01-02,Withdrawal 25991738 Free Interac E Transfer,800.0,NaN,1401.0
2,2025-01-02,MB Transfer from 232001691155,NaN,6000.0,7401.0
3,2025-01-02,Point of sale purchase Apos Driver Services Va...,1500.0,NaN,5901.0
4,2025-01-02,Point of sale purchase Apos Driver Services Va...,1000.0,NaN,4901.0


In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         618 non-null    object 
 1   description  618 non-null    object 
 2   withdrawals  484 non-null    float64
 3   deposits     109 non-null    float64
 4   balance      618 non-null    float64
dtypes: float64(3), object(2)
memory usage: 24.3+ KB


In [20]:
df_clean.loc[df_clean['description'] == 'Opening Balance'].shape

(0, 5)

In [21]:
df_clean.loc[df_clean['description'] == 'Closing Balance'].shape

(0, 5)

In [19]:
df_clean = df_clean.drop(df_clean.loc[df_clean['description'] == 'Opening Balance'].index)
df_clean = df_clean.drop(df_clean.loc[df_clean['description'] == 'Closing Balance'].index)

In [22]:
df_clean['transaction_type'] = df_clean.apply(
    lambda row: 'withdrawal' if pd.notna(row['withdrawals']) and row['withdrawals'] > 0
    else 'deposit' if pd.notna(row['deposits']) and row['deposits'] > 0
    else 'unknown', axis=1
)

In [25]:
df_clean['transaction_type'].value_counts()

transaction_type
withdrawal    484
deposit       108
unknown         1
Name: count, dtype: int64

In [26]:
df_clean.loc[df_clean['transaction_type'] == 'unknown']

,date,description,withdrawals,deposits,balance,transaction_type
180,2024-08-02,Deposit,NaN,0.0,0.0,unknown


In [31]:
df_clean['deposits'].sum()

np.float64(3133656.0)

In [32]:
df_clean['withdrawals'].sum()

np.float64(8122158.0)

In [33]:
df_clean.head()

,date,description,withdrawals,deposits,balance,transaction_type
1,2025-01-02,Withdrawal 25991738 Free Interac E Transfer,800.0,NaN,1401.0,withdrawal
2,2025-01-02,MB Transfer from 232001691155,NaN,6000.0,7401.0,deposit
3,2025-01-02,Point of sale purchase Apos Driver Services Va...,1500.0,NaN,5901.0,withdrawal
4,2025-01-02,Point of sale purchase Apos Driver Services Va...,1000.0,NaN,4901.0,withdrawal
5,2025-01-02,Point of sale purchase Apos Sal army West Br V...,4635.0,NaN,266.0,withdrawal


#### Feature Engineering

In [35]:
df_clean['date'] = pd.to_datetime(df_clean['date'])

df_clean['day'] = df_clean['date'].dt.day
df_clean['month'] = df_clean['date'].dt.month
df_clean['year'] = df_clean['date'].dt.year
df_clean['day_of_week'] = df_clean['date'].dt.dayofweek

In [38]:
print(df_clean.sample(10))

          date                                        description  \
26  2025-01-11  Point of sale purchase Apos Dollar ama 1373 Va...   
334 2025-06-30                      MB Transfer from 232001691155   
202 2024-08-07  Error correction F posTl nk 0453008319 Burnaby...   
281 2024-09-20  Point of sale purchase Op osI c Insta cart 188...   
344 2025-07-02  Point of sale purchase Op o sAm zn Mkt pCA Www...   
358 2025-07-03       Point of sale purchase A posTl nk 0484914343   
226 2024-08-16  Point of sale purchase Coin amati c 2707 Tenni...   
372 2025-07-07  Point of sale purchase O posA BC 803 Golds Gym...   
465 2025-05-22                      MB Transfer from 232001691155   
143 2024-10-01        Withdrawal 84938279 Free Interac E Transfer   

     withdrawals  deposits   balance transaction_type  day  month  year  \
26        2440.0       NaN    8174.0       withdrawal   11      1  2025   
334          NaN    5000.0    8117.0          deposit   30      6  2025   
202        635.

In [39]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 593 entries, 1 to 616
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              593 non-null    datetime64[ns]
 1   description       593 non-null    object        
 2   withdrawals       484 non-null    float64       
 3   deposits          109 non-null    float64       
 4   balance           593 non-null    float64       
 5   transaction_type  593 non-null    object        
 6   day               593 non-null    int32         
 7   month             593 non-null    int32         
 8   year              593 non-null    int32         
 9   day_of_week       593 non-null    int32         
dtypes: datetime64[ns](1), float64(3), int32(4), object(2)
memory usage: 41.7+ KB


In [40]:
df_clean.to_csv(f"{PROCESSED_PATH}/df_v2.csv", index=False)